In [ ]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

import json
import pymongo as pm

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [ ]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_group_data_for_annotation.csv'
meta = pd.read_csv(path_to_metadata)    

In [ ]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append('"leg","arm","back","seat","wheels","head"')
    if meta.category[i]=="dog":
        parts.append('"eye","mouth","ear","body","tail","neck","leg"')
    if meta.category[i] == "bird":
        parts.append('"beak","wings","feet","tail","body","head"')
    if meta.category[i] == "car":
        parts.append('"wheel", "windshield", "body", "window", "door", "bumper", "lamp"')
parts = pd.Series(parts)
meta = meta.assign(parts=pd.Series(parts))

In [ ]:
## add iteration name information
_iterationName = 'sketchpad_basic'
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [ ]:
## svg string formatting
meta['svg'] = meta['svg'].str.replace('u', '')
meta['svg'] = meta['svg'].str.replace("'",'"') 

In [ ]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [ ]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta.js',mode='rU').read())
assert len(J)==len(meta)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [ ]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic']

In [ ]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = False
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)

In [ ]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))